In [1]:
from binance import Client
import requests
from dotenv import load_dotenv
import os
import ta
import pandas as pd
from time import sleep
from binance.exceptions import BinanceAPIException
import time

In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv("BINANCE_API_KEY")
api_secret = os.getenv("BINANCE_SECRET_KEY")

In [4]:
client = Client(api_key, api_secret)

In [5]:
pd.DataFrame(client.get_historical_klines('ETHBUSD', '1m', '10m ago UTC'))

,0,1,2,3,4,5,6,7,8,9,10,11
0,1639016700000,4391.02000000,4398.00000000,4385.67000000,4385.67000000,112.80960000,1639016759999,495664.21513500,371,38.34820000,168554.93643700,0
1,1639016760000,4385.67000000,4391.12000000,4384.53000000,4386.08000000,106.08330000,1639016819999,465390.52756000,271,68.04040000,298463.42375100,0
2,1639016820000,4386.32000000,4392.95000000,4386.21000000,4392.42000000,68.97450000,1639016879999,302682.64663000,220,22.80210000,100052.73676300,0
3,1639016880000,4392.41000000,4394.23000000,4387.65000000,4393.01000000,62.66220000,1639016939999,275241.43537500,216,27.69330000,121640.58883900,0
4,1639016940000,4393.01000000,4394.23000000,4390.21000000,4393.76000000,30.50980000,1639016999999,134024.95287800,147,10.89600000,47859.58543900,0
5,1639017000000,4393.45000000,4394.31000000,4385.01000000,4394.31000000,82.51610000,1639017059999,362128.78688500,297,44.11070000,193610.11612800,0
6,1639017060000,4394.49000000,4403.20000000,4392.86000000,4397.19000000,234.45230000,1639017119999,1031212.78141100,730,118.32120000,520406.61654700,0
7,1639017120000,4397.59000000,4401.18000000,4397.59000000,4400.67000000,70.01350000,1639017179999,308062.79532300,162,50.03610000,220151.34502200,0
8,1639017180000,4400.81000000,4401.31000000,4397.36000000,4398.73000000,26.44180000,1639017239999,116330.91893100,143,9.08460000,39967.24198700,0
9,1639017240000,4398.73000000,4400.75000000,4394.51000000,4394.51000000,55.54350000,1639017299999,244239.93191700,144,11.51370000,50642.77071200,0


In [6]:
def getdata(symbol):
    try:
        frame = pd.DataFrame(client.get_historical_klines(symbol, '1m', '10m UTC'))
    except BinanceAPIException as e:
        print(e)
        sleep(60)
        frame = pd.DataFrame(client.get_historical_klines(symbol, '1m', '10m UTC'))
    frame = frame.iloc[:, :6]
    frame.columns = ['Time','Open', 'High','Low','Close','Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame

# def getminutedata(symbol):
#     frame = pd.DataFrame(client.get_historical_klines(symbol, '1m', '40m UTC'))
#     frame = frame.iloc[:, :6]
#     frame.columns = ['Time','Open', 'High','Low','Close','Volume']
#     frame = frame.set_index('Time')
#     frame.index = pd.to_datetime(frame.index, unit='ms')
#     frame = frame.astype(float)
#     return frame
                               

In [7]:
getdata('ETHBUSD')

,Open,High,Low,Close,Volume
Time,,,,,
2021-12-09 02:25:00,4391.02,4398.00,4385.67,4385.67,112.8096
2021-12-09 02:26:00,4385.67,4391.12,4384.53,4386.08,106.0833
2021-12-09 02:27:00,4386.32,4392.95,4386.21,4392.42,68.9745
2021-12-09 02:28:00,4392.41,4394.23,4387.65,4393.01,62.6622
2021-12-09 02:29:00,4393.01,4394.23,4390.21,4393.76,30.5098
2021-12-09 02:30:00,4393.45,4394.31,4385.01,4394.31,82.5161
2021-12-09 02:31:00,4394.49,4403.20,4392.86,4397.19,234.4523
2021-12-09 02:32:00,4397.59,4401.18,4397.59,4400.67,70.0135
2021-12-09 02:33:00,4400.81,4401.31,4397.36,4398.73,26.4418


In [8]:
def tradingstrat(symbol, qty, open_position = False):
    while True:
        df = getdata(symbol)
        print(f'current close is $' +str(df.Close.iloc[-1]))
        if not open_position:
            if ta.trend.macd_diff(df.Close).iloc[-1] > 0 \
            and ta.trend.macd_diff(df.Close).iloc[-2] < 0:
                order = client.create_order(symbol=symbol,
                                               side='BUY',
                                               type='MARKET',
                                               quantity=qty)
                print(order)
                open_position = True
                buyprice = float(order['fills'][0]['price'])
                break 
                
        if open_position:
            while True:
                df = getdata(symbol)
                
                if ta.trend.macd_diff(df.Close).iloc[-1] < 0 \
                and ta.trend.macd_diff(df.Close).iloc[-2] > 0 | df.Close[-1] <= buyprice * 0.90 or df.Close >= 2 * buyprice:
                    order = client.create_order(symbol=symbol,
                                               side='BUY',
                                               type='MARKET',
                                               quantity=qty) 
                    print(order)
                    sellprice = float(order['fills'][0]['price'])
                    print(f'profit = {(sellprice - buyprice)/buyprice}')
                    open_position = False
                    break

In [ ]:
while True:
    tradingstrat('ETHBUSD', qty=0.010)
    

current close is $4393.87
current close is $4393.87
current close is $4393.54
current close is $4393.54
current close is $4393.54
current close is $4393.87
current close is $4393.66
current close is $4393.66
current close is $4393.66
current close is $4394.12
current close is $4394.12
current close is $4393.27
current close is $4393.27
current close is $4393.27
current close is $4393.27
current close is $4393.27
current close is $4393.27
current close is $4393.27
current close is $4393.27
current close is $4393.64
current close is $4393.64
current close is $4393.64
current close is $4393.64
current close is $4393.64
current close is $4393.64
current close is $4393.64
current close is $4393.64
current close is $4393.64
current close is $4393.64
current close is $4393.64
current close is $4393.64
current close is $4394.12
current close is $4394.12
current close is $4394.12
current close is $4394.12
current close is $4394.12
current close is $4394.12
current close is $4394.12
current clos

In [ ]:
# APIError(code=-1001): Internal error; unable to process your request. Please try again.